https://stepik.org/lesson/1576182/step/8

In [1]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim

class FuncModel(nn.Module):
    def __init__(self):
        super().__init__()
        # модель однослойной полносвязной нейронной сети:
        # 1-й слой: число входов 5 (x, x^2, x^3, x^4, x^5), число нейронов 1
        self.layer = nn.Linear(5, 1)

    def forward(self, x):

        # x.unsqueeze_(-1)
        # xx = torch.cat([x, x ** 2, x ** 3, x ** 4, x ** 5], dim=1)

        # то же самое через stack в одну строку
        xx = torch.stack([x, x ** 2, x ** 3, x ** 4, x ** 5], dim=1)

        y = self.layer(xx)
        return y


torch.manual_seed(1)

model = FuncModel() # создать модель FuncModel

epochs = 20         # число эпох обучения
batch_size = 16     # размер батча

# Данные обучающей выборки (значения функции)
data_x = torch.arange(-5, 5, 0.05) #тензоры data_x, data_y не менять
data_y = torch.sin(2 * data_x) - 0.3 * torch.cos(8 * data_x) + 0.1 * data_x ** 2

# Создание dataset
ds = data.TensorDataset(data_x, data_y)
d_train, d_val = data.random_split(ds, [0.7, 0.3])
train_data = data.DataLoader(d_train, batch_size=batch_size, shuffle=True)
train_data_val = data.DataLoader(d_val, batch_size=batch_size, shuffle=False)

# Оптимизатор и функция потерь
optimizer = optim.RMSprop(params=model.parameters(), lr=0.01)
loss_func =  torch.nn.MSELoss()

loss_lst_val = []  # список значений потерь при валидации
loss_lst = []  # список значений потерь при обучении

# Градиентный спуск:
for _e in range(epochs):
    # перевод модели в режим обучения
    model.train()
    # вспомогательные переменные для вычисления среднего значения потерь при обучении:
    loss_mean = 0
    lm_count = 0

    for x_train, y_train in train_data:
        predict = model(x_train).squeeze() # вычислить прогноз модели для данных x_train
        loss = loss_func(predict, y_train) # вычислить значение функции потерь

        # сделать один шаг градиентного спуска для корректировки параметров модели
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # вычисление среднего значения функции потерь по всей выборке
        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean

    # Проверка качества модели
    model.eval() # перевод модели в режим эксплуатации
    # сохранить средние потери, вычисленные по выборке валидации, в переменной Q_val
    Q_val = 0
    count_val = 0

    for x_val, y_val in train_data_val:
        with torch.no_grad():
            # для x_val, y_val вычислить потери с помощью функции loss_func
            pred = model(x_val).squeeze()   # Прогноз
            loss = loss_func(pred, y_val)   # Вычисление потерь
            Q_val += loss.item()
            count_val +=1

    loss_lst.append(loss_mean)
    loss_lst_val.append(Q_val / count_val)

# Перевод модель в режим эксплуатации
model.eval()

# Прогноз модели по всем данным выборки (ds.data)
predict = model(data_x).squeeze()

# Потери с помощью loss_func по всем данным выборки ds; значение Q сохранить в виде вещественного числа
Q = loss_func(predict, data_y).item()

print(Q)
print(loss_lst)
print(loss_lst_val)

17.375396728515625
[58000.0390625, 3705.9929131401914, 430.9513530731201, 85.20848337809245, 34.44961049821642, 23.521960682339138, 19.50560885005528, 18.379872534010147, 18.04986233181424, 17.378559960259334, 17.50553406609429, 17.526691648695202, 17.30922285715739, 17.085827933417427, 16.803658379448787, 17.058402379353844, 16.40661409166124, 15.860542880164251, 16.22151358922323, 15.998286777072483]
[10337.098510742188, 1151.7630157470703, 192.67199766635895, 64.51006650924683, 38.43087816238403, 29.902281284332275, 27.430378913879395, 26.283755779266357, 25.67999267578125, 25.273123264312744, 24.962718963623047, 24.793481826782227, 24.577940940856934, 24.740113735198975, 24.173279762268066, 23.386428833007812, 23.45594596862793, 23.074254035949707, 23.51522445678711, 22.78296995162964]
